In [ ]:
# Import all necessart packages

from bokeh.models import CustomJS, ColumnDataSource, BoxSelectTool, Range1d, Rect,Div, Button
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column, row
from IPython.display import clear_output, display
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box
import numpy as np
import cv2
import os
import sys

output_notebook()

In [ ]:
def _reset_keeping_doc(self):
    ''' Reset output modes but DO NOT replace the default Document'''
    self._file = None
    self._notebook = False
    self._notebook_type = None

class AnnotationProject(object):
    
    def __init__(self, basePath=None):
        self.BoxList = list()
    
        # Files to look for
        self.imageExtensions = ['jpg', 'bmp', 'jpeg', 'png']

        self.projectPath = dict()
        if basePath is None:
            self.projectPath['BASE'] = os.path.join(os.environ['HOME'], 'CV_AnnotationProject')
        else:
            self.projectPath['BASE'] = basePath
        self.projectPath['IMAGES'] = os.path.join(self.projectPath['BASE'], 'images')
        self.projectPath['ANNOTATIONS'] = os.path.join(self.projectPath['BASE'], 'annotations')
        
        # Images access from project
        self.imageDatabase = dict()
        self.imageList = list()
               
        # Check directory structure is setup
        self.checkSetup()
        self.createImageLabels()
        
        # Initialize Bokeh
        self.initializeBokeh()
    
    def initializeBokeh(self):
        self.source = ColumnDataSource(data=dict(x=[], y=[], width=[], height=[]))

        callback = CustomJS(args=dict(source=self.source), code="""
            // get data source from Callback args
            var data = source.data;

            /// get BoxSelectTool dimensions from cb_data parameter of Callback
            var geometry = cb_data['geometry'];

            /// calculate Rect attributes
            var width = geometry['x1'] - geometry['x0'];
            var height = geometry['y1'] - geometry['y0'];
            var x = geometry['x0'] + width/2;
            var y = geometry['y0'] + height/2;

            /// update data source with new Rect attributes
            var x0 = geometry['x0'];
            var y0 = geometry['y0'];
            data['x'].push(x);
            data['y'].push(y);
            data['width'].push(width);
            data['height'].push(height);

            // emit update of data source
            source.change.emit();
        
            // Communicate with IPython
            var kernel = IPython.notebook.kernel;
            var coord = x0 + ", " + y0 + ", " + width + ", " + height;
            var command1 = "bbox = np.array([" + coord + "])";
            var command2 = "annotateImage.BoxList.append(bbox)";
        
            kernel.execute(command1);
            kernel.execute(command2);
            """)

        self.box_select = BoxSelectTool(callback=callback)
    
    def generateBokehDisplay(self, img):
        
        h_img = img.shape[0]
        w_img = img.shape[1]
        self.p = figure(plot_width=w_img,
               plot_height=h_img,
               tools=[self.box_select],
               title="Select Below",
               x_range=Range1d(start=0.0, end=w_img),
               y_range=Range1d(start=0.0, end=h_img))
    
        self.p.image(image=[img[::-1, :]], x=[0], y=[0],
            dw=[w_img], dh=[h_img], palette="Greys9")
 
        rect = Rect(x='x',y='y',width='width',height='height',fill_alpha=0.3,fill_color='#009933')
        self.p.add_glyph(self.source, rect, selection_glyph=rect, nonselection_glyph=rect)
    
        # Generate Bokeh display
        buttonList = ['Clear','Show','RejectImage','NextImage']
        buttonTupleList = [(widgets.Button(description=name),name) for name in buttonList]
        buttonItems = [button for button,name in buttonTupleList]
        boxLayout = Layout(display='flex',
                        flex_flow='row',
                        align_items='stretch',
                        border='solid',
                        width='45%')
        self.box = Box(children=buttonItems, layout=boxLayout)
        buttonClicks = [button.on_click(getattr(AnnotationProject, 'func'+name)) for button,name in buttonTupleList]
        display(self.box)
        show(self.p)
                
    def runAnnotator(self):
        if self.imageList:
            self.BoxList = list()
            self.correctLabel = True
            
            imageFile = self.imageList[0]
            print ('Annotating ', imageFile)
            img = cv2.imread(self.imageDatabase[imageFile][0])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            self.currentImg = img
            self.generateBokehDisplay(img)   
        else:
            print ('No more images left to annotate')

    def funcClear(self):
        global annotateImage
        clear_output()
        _reset_keeping_doc(annotateImage.p)
        
        #Reset the information captures
        annotateImage.BoxList = list()
        annotateImage.correctLabel = True
        annotateImage.runAnnotator()
        
    def createStringCoordinates(self, x,y,widthBox,heightBox,widthImage,heightImage):
        # VOC: <class> <bboxXMin> <bboxYMin> <bboxXMax> <bboxYMax>
        # Raw coordinates
        bboxString = 'x0 ' + str(int(x)) + ' y0 ' + str(int(y)) + ' w ' + str(int(x) +int(widthBox)) + ' h ' + str(int(y) + int(heightBox))
        bboxString += ' , '
        # Centered coordinates of YOLO : <centerX/imageWidth> <centerY/imageHeight> <bboxWidth/imageWidth> <bboxHeight/imageHeight>
        cx = (x+widthBox/2)
        cy = (y+heightBox/2)
        bboxString += str(cx/widthImage) + ' ' + str(cy/heightImage) + ' ' + str(widthBox/widthImage) + ' ' + str(heightBox/heightImage)
        bboxString += '\n'
        return bboxString
                                                                 
                                                                 
    
    def funcShow(self):
        global annotateImage
        heightImage = annotateImage.currentImg.shape[0]
        widthImage = annotateImage.currentImg.shape[1]
        print ('There are ' + str(len(annotateImage.BoxList)) + ' boxes on this image')
        print ('The current image is (width,height) = ', widthImage, heightImage)
        print ('Status of image correct label : ', str(annotateImage.correctLabel))
        print ('The coordinates of bounding boxes on the image are: \n')
        for item in np.arange(0, len(annotateImage.BoxList)):
            print ('Box '+str(item))
            x = int(annotateImage.BoxList[item][0])
            y = int(annotateImage.BoxList[item][1])
            widthBox = int(annotateImage.BoxList[item][2])
            heightBox = int(annotateImage.BoxList[item][3])
            print ('x0,y0,w,h   : ',x,y,widthBox,heightBox)
            print ('x0,y0,x1,y1 : ',x,y,x+widthBox,y+heightBox)
            print ('cx,cy       : ',x+widthBox/2, y+heightBox/2)
            print ('cx/w cy/h w/imgw h/imgh', (x+widthBox/2)/widthBox, (y+heightBox/2)/heightBox, widthBox/widthImage, heightBox/heightImage)
            
    def funcSave(self):
        # Get YOLO Weights
        lineTemp = np.zeros((len(self.BoxList), 4))
        
        heightImage = self.currentImg.shape[0]
        widthImage = self.currentImg.shape[1]
        
        # Yolo: <class> <centerX/imageWidth> <centerY/imageHeight> <bboxWidth/imageWidth> <bboxHeight/imageHeight>
        # VOC: <class> <bboxXMin> <bboxYMin> <bboxXMax> <bboxYMax>
        
        for row in np.arange(len(self.BoxList)):
            x = self.BoxList[row][0]
            y = self.BoxList[row][1]
            widthBox = self.BoxList[row][2]
            heightBox = self.BoxList[row][3]
            bboxString = self.createStringCoordinates(x,y,widthBox,heightBox,widthImage,heightImage)
            print ('bbox string ', bboxString)
        
        #write to file   +
        print ('Writing to file ' + self.imageDatabase[self.imageList[0]][1])
        fp = open(self.imageDatabase[self.imageList[0]][1], 'w')
        imageClass = self.imageDatabase[self.imageList[0]][2]
        imageNumber = self.imageDatabase[self.imageList[0]][3]
        correctLabel = str(self.correctLabel)
        

        # Define first line of image annotation        
        outputString = ' '.join([imageClass,imageNumber,correctLabel, 'H:', str(heightImage), 'W:', str(widthImage),'\n'])
    
        # Define a new line for each boxed detection
        for row in np.arange(len(self.BoxList)):
            x = self.BoxList[row][0]
            y = self.BoxList[row][1]
            widthBox = self.BoxList[row][2]
            heightBox = self.BoxList[row][3]
            bboxString = self.createStringCoordinates(x,y,widthBox,heightBox,widthImage,heightImage)
            outputString += bboxString
    
        # Write to output
        fp.write(outputString)
        fp.close()
        

    def funcNextImage(self):
        global annotateImage
        clear_output()
        _reset_keeping_doc(annotateImage.p)
        
        annotateImage.funcSave()
        del annotateImage.imageList[0]
        annotateImage.BoxList = list()
        
        # Get new image from dictionary
        annotateImage.runAnnotator()
        
        
    def funcRejectImage(self):
        global annotateImage
        annotateImage.correctLabel = False

    def checkSetup(self):
        if not os.path.exists(self.projectPath['BASE']):
            print ('No project directory')
            sys.exit(-1)
    
        if not os.path.exists(self.projectPath['IMAGES']):
            print ('No image directory under project')
            sys.exit(-1)

        if not os.path.exists(self.projectPath['ANNOTATIONS']):
            os.makedirs(self.projectPath['ANNOTATIONS'])

    def createImageLabels(self):
        # Generate a base list of files
        fileList = [f for f in os.listdir(os.path.join(self.projectPath['IMAGES'])) if os.path.isfile(os.path.join(os.path.join(self.projectPath['IMAGES'], f))) and f.lower().endswith(tuple(self.imageExtensions))]
        print ('There are ', len(fileList), ' image files')
        
        # Only work on images that do not have Annotations
        self.imageList = list()
        for imageFile in fileList:
            annotationFile = os.path.splitext(imageFile)[0]+'.txt'
            imageClass = os.path.splitext(imageFile)[0].split('_')[0]
            imageNumber = os.path.splitext(imageFile)[0].split('_')[1]
            if not os.path.isfile(os.path.join(self.projectPath['ANNOTATIONS'], annotationFile)): 
                self.imageDatabase[imageFile] = (os.path.join(self.projectPath['IMAGES'], imageFile), 
                                                 os.path.join(self.projectPath['ANNOTATIONS'], annotationFile),
                                                imageClass, imageNumber)

                self.imageList.append(imageFile);[]
        #print (self.imageDatabase)
        print ('There are ', len(self.imageList), ' image files to annotate')
  

In [ ]:
annotateImage = AnnotationProject()
annotateImage.runAnnotator()
